In [47]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import ipywidgets as widgets
from ipywidgets import interact

In [60]:
class bola:
    def __init__(self, x, y, vx, vy, r):
        self.x = [x]
        self.y = [y]
        self.vx = [vx]
        self.vy = [vy]
        self.r = r

xi, xf = -5, 5
yi, yf = -5, 5
n = 5
        
b = [bola(np.random.uniform(xi, xf), 
          np.random.uniform(yi, yf),
          np.random.normal(1.0, 0.5),
          np.random.normal(1.0, 0.5),
          np.random.normal(0.5, 0.1)) for i in range(n)]

In [61]:
maxiter = 100
dt = 0.1

for i in range(maxiter):
    
    # atualização das posições na dinâmica
    for j in b:
        j.x.append(j.x[-1] + dt * j.vx[-1])
        j.y.append(j.y[-1] + dt * j.vy[-1])
    
    # atualização das velocidades. Temos diversas situações aqui
    # (1) choque entre duas esferas
    for j in range(n):
        for k in range(j + 1, n):
            if np.sqrt((b[j].x[-1] - b[k].x[-1]) ** 2 + (b[j].y[-1] - b[k].y[-1]) ** 2) < (b[j].r + b[k].r):
                t = np.arctan((b[k].y[-1] - b[j].y[-1]) / (b[k].x[-1] - b[j].x[-1]))
                v = np.sqrt(b[j].vx[-1] ** 2 + b[j].vy[-1] ** 2)
                p = np.random.normal(np.pi + t, np.pi / 6)
                b[j].vx.append(v * np.cos(t))
                b[j].vy.append(v * np.sin(t))
            
    
    for j in b:
        # (2) esfera bate na parede da esquerda
        if np.abs(j.x[-1] - xi) < j.r:
            t = np.random.normal(0, np.pi / 6)
            v = np.sqrt(j.vx[-1] ** 2 + j.vy[-1] ** 2)
            j.vx.append(v * np.cos(t))
            j.vy.append(v * np.sin(t))
        # (3) esfera bate na parede da direita
        if np.abs(j.x[-1] - xf) < j.r:
            t = np.random.normal(np.pi, np.pi / 6)
            v = np.sqrt(j.vx[-1] ** 2 + j.vy[-1] ** 2)
            j.vx.append(v * np.cos(t))
            j.vy.append(v * np.sin(t))
        # (4) esfera bate na parede superior
        if np.abs(j.y[-1] - yi) < j.r:
            t = np.random.normal(np.pi / 2, np.pi / 6)
            v = np.sqrt(j.vx[-1] ** 2 + j.vy[-1] ** 2)
            j.vx.append(v * np.cos(t))
            j.vy.append(v * np.sin(t))
        # (5) esfera bate na parede inferior
        if np.abs(j.y[-1] - yf) < j.r:
            t = np.random.normal(- np.pi / 2, np.pi / 6)
            v = np.sqrt(j.vx[-1] ** 2 + j.vy[-1] ** 2)
            j.vx.append(v * np.cos(t))
            j.vy.append(v * np.sin(t))
    
    # (6) não bate em nada
    for j in b:
        if len(j.vx) == i + 1:
            j.vx.append(j.vx[-1])
            j.vy.append(j.vy[-1])
#         print(i, len(j.vx), len(j.vy))

In [57]:
for i in range(maxiter):
    print('{:3d} : {:6.3f}, {:6.3f}, {:6.3f}, {:6.3f}'.format(i, b[0].x[i], b[0].y[i], b[0].x[i], b[0].y[i]))

  0 :  2.676, -3.803,  2.676, -3.803
  1 :  2.664, -3.680,  2.664, -3.680
  2 :  2.652, -3.558,  2.652, -3.558
  3 :  2.640, -3.435,  2.640, -3.435
  4 :  2.628, -3.312,  2.628, -3.312
  5 :  2.615, -3.190,  2.615, -3.190
  6 :  2.603, -3.067,  2.603, -3.067
  7 :  2.591, -2.944,  2.591, -2.944
  8 :  2.579, -2.822,  2.579, -2.822
  9 :  2.566, -2.699,  2.566, -2.699
 10 :  2.554, -2.577,  2.554, -2.577
 11 :  2.542, -2.454,  2.542, -2.454
 12 :  2.530, -2.331,  2.530, -2.331
 13 :  2.517, -2.209,  2.517, -2.209
 14 :  2.505, -2.086,  2.505, -2.086
 15 :  2.493, -1.964,  2.493, -1.964
 16 :  2.481, -1.841,  2.481, -1.841
 17 :  2.468, -1.718,  2.468, -1.718
 18 :  2.456, -1.596,  2.456, -1.596
 19 :  2.444, -1.473,  2.444, -1.473
 20 :  2.432, -1.350,  2.432, -1.350
 21 :  2.420, -1.228,  2.420, -1.228
 22 :  2.407, -1.105,  2.407, -1.105
 23 :  2.395, -0.983,  2.395, -0.983
 24 :  2.383, -0.860,  2.383, -0.860
 25 :  2.371, -0.737,  2.371, -0.737
 26 :  2.416, -0.623,  2.416, -0.623
 

In [63]:
@interact(i=(0, maxiter - 1))

def showme(i):
    fig = plt.figure(figsize=(8,8))
    ax = fig.gca()
    
    circles = [plt.Circle((j.x[i], j.y[i]), radius=j.r, linewidth=0) for j in b]
#                plt.Circle((bB.x[i],bB.y[i]), radius=.75, linewidth=0, color='blue')]
    c = matplotlib.collections.PatchCollection(circles)
    ax.add_collection(c)
#     ax.scatter(bA.x[i], bA.y[i], s=200, color='red')
#     ax.scatter(bB.x[i], bB.y[i], s=200, color='blue')
    ax.set_title('step {:3d}'.format(i))
    ax.set_xlim(xi, xf)
    ax.set_ylim(yi, yf)
    
    plt.show()

interactive(children=(IntSlider(value=49, description='i', max=99), Output()), _dom_classes=('widget-interact'…